In [34]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
from seaborn import set_style

## This sets the plot style
## to have a grid on a white background
set_style("darkgrid")

In [35]:
X = np.load('../../data/processed_data/specgram_db_pad_trunc.npy')
print(X.shape)

(440, 128, 67)


In [36]:
# NEW to CONV nets
## We need to reshape our data to be the correct dimensions for
## the net
## observations, grid, colorvalues

##so we just need to add the colorvalues, 1 here represents single valued
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
print(X.shape)

(440, 128, 67, 1)


In [37]:
df = pd.read_csv("../../data/processed_data/metadata.csv")
category_to_number = {'brushing': 0, 'food': 1, 'isolation': 2}

# Create a new column with numerical values based on the mapping
df['numerical_situation'] = df['situation'].map(category_to_number)

y = df['numerical_situation']


In [38]:
df

,Unnamed: 0,filename,situation,sex,breed,numerical_situation
0,0,F_MAG01_EU_FN_FED01_106.wav,food,0,european_shorthair,1
1,1,I_NUL01_MC_MI_SIM01_304.wav,isolation,1,maine_coon,2
2,2,I_MEG01_MC_FI_SIM01_304.wav,isolation,0,maine_coon,2
3,3,B_CAN01_EU_FN_GIA01_205.wav,brushing,0,european_shorthair,0
4,4,F_MAG01_EU_FN_FED01_304.wav,food,0,european_shorthair,1
...,...,...,...,...,...,...
435,435,I_TIG01_EU_FN_GIU01_201.wav,isolation,0,european_shorthair,2
436,436,B_NIG01_MC_MN_SIM01_101.wav,brushing,1,maine_coon,0
437,437,B_CAN01_EU_FN_GIA01_208.wav,brushing,0,european_shorthair,0
438,438,B_WHO01_MC_FI_SIM01_203.wav,brushing,0,maine_coon,0


In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                       test_size=.2,
                                                       shuffle=True,
                                                       random_state=100,
                                                       stratify=y)

In [40]:
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train, y_train,
                                                                 test_size=.2,
                                                                   shuffle=True,
                                                                   random_state=41,
                                                                   stratify=y_train)

In [41]:
## Importing the things
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.utils import to_categorical

# for earlier versions of keras run this instead
# from keras.utils import to_categorical
# instead of running
# from keras.utils.np_utils import to_categorical
# or look up the documentation for your version

In [42]:
## We've seen this before
model = models.Sequential()

In [43]:
## adding our first convolutional layer

## Conv2D(32, specifies that we want a convolutional layer with depth 32
## (3,3) is our sliding grid size
## We're using the relu activation function
## and our images are a gray scale value for 28X28 pixels, hence the input_shape
model.add( layers.Conv2D(32, (6,3), activation='relu', input_shape=(X.shape[1],X.shape[2],1,) ) )


## adding our first Max Pooling layer
## MaxPooling2D((2,2) tells python we want to add
## a max pooling layer that uses a (2,2) square grid
## strides = 2, sets the stride value to 2
model.add( layers.MaxPooling2D( (2,2), strides = 2 ) )


## Add another layer, alternating between conv and pool
model.add( layers.Conv2D(64, (6,3), activation='relu'))
model.add( layers.MaxPooling2D( (2,2), strides=2) )



In [44]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 123, 65, 32)       608       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 61, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 56, 30, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 28, 15, 64)        0         
 g2D)                                                            
                                                                 
Total params: 37536 (146.62 KB)
Trainable params: 37536 (146.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
## Now we'll add the fully connected layer

## .Flatten() will flatten the data for us,
## meaning the last output data will turn into 
## a vector
model.add( layers.Flatten() )

## We've seen these before
## Then we add a single dense hidden layer
## This is 64 nodes high
model.add(layers.Dense(64, activation='relu'))


## Finally an output layer
model.add(layers.Dense(3, activation='softmax'))

In [46]:
## We've seen this before in the last notebook
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
## we'll go for 10 epochs
## This is slow!
epochs=100
history = model.fit(X_train_train, 
                      to_categorical(y_train_train), 
                      epochs=epochs, 
                      batch_size=512,
                      validation_data=(X_val,to_categorical(y_val)))

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 13.6317 - accuracy: 0.2883 - val_loss: 230.2918 - val_accuracy: 0.5070
Epoch 2/100
1/1 [==============================] - 1s 924ms/step - loss: 237.7491 - accuracy: 0.5018 - val_loss: 88.0081 - val_accuracy: 0.2113
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 88.4721 - accuracy: 0.2100 - val_loss: 3.6023 - val_accuracy: 0.5070
Epoch 4/100
1/1 [==============================] - 1s 940ms/step - loss: 3.7070 - accuracy: 0.5018 - val_loss: 1.8478 - val_accuracy: 0.2817
Epoch 5/100
1/1 [==============================] - 1s 903ms/step - loss: 1.8144 - accuracy: 0.2883 - val_loss: 1.0341 - val_accuracy: 0.5493
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.9819 - accuracy: 0.5516 - val_loss: 1.8626 - val_accuracy: 0.5070
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 1.8037 - accuracy: 0.5089 - val_loss: 0.9699 - val_accuracy: 0.5915
Epoch 8/100
1/1 [=

Epoch 59/100
1/1 [==============================] - 1s 931ms/step - loss: 2.0627 - accuracy: 0.6655 - val_loss: 5.9054 - val_accuracy: 0.5352
Epoch 60/100
1/1 [==============================] - 1s 929ms/step - loss: 3.6877 - accuracy: 0.6548 - val_loss: 29.7707 - val_accuracy: 0.2817
Epoch 61/100
1/1 [==============================] - 1s 932ms/step - loss: 28.7067 - accuracy: 0.2883 - val_loss: 2.2397 - val_accuracy: 0.5915
Epoch 62/100
1/1 [==============================] - 1s 949ms/step - loss: 1.0262 - accuracy: 0.7011 - val_loss: 10.9575 - val_accuracy: 0.3521
Epoch 63/100
1/1 [==============================] - 1s 889ms/step - loss: 10.5875 - accuracy: 0.4413 - val_loss: 3.0212 - val_accuracy: 0.5634
Epoch 64/100
1/1 [==============================] - 1s 935ms/step - loss: 0.7778 - accuracy: 0.6868 - val_loss: 1.5624 - val_accuracy: 0.6761
Epoch 65/100
1/1 [==============================] - 1s 936ms/step - loss: 0.2476 - accuracy: 0.9146 - val_loss: 1.4442 - val_accuracy: 0.7042
Ep

In [ ]:
history_dict = history.history

In [ ]:
plt.figure(figsize = (8,6))

plt.scatter(range(1,epochs+1), history_dict['accuracy'], label = "Training Accuracy")
plt.scatter(range(1,epochs+1), history_dict['val_accuracy'], label = "Validation Set Accuracy")

plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.legend(fontsize=12)

plt.show()


plt.figure(figsize = (8,6))

plt.scatter(range(1,epochs+1), history_dict['loss'], label = "Training Loss")
plt.scatter(range(1,epochs+1), history_dict['val_loss'], label = "Validation Set Loss")

plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Loss Function Value", fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.legend(fontsize=12)

plt.show()